# Import RPB model along with other utility functions

In [1]:
from RPB_model import *

expression symbols from pyomo.core.expr  (deprecated in 6.6.2.dev0) (called
from <frozen importlib._bootstrap>:241)


# Creating model for a single side/section of the RPB

In [ ]:
# Create model instance. Currently, mode can be either "adsorption" or "desorption" which sets the boundary conditions for each case.
# m=RPB_model(mode="adsorption")
# or
m=RPB_model(mode="desorption")

# Custom initialization routine. Uses a homotopy routine to adjust the initialization factors so that all equations are 'active'.
single_section_init(m)

In [ ]:
# Plotting the results/profiles

plotting(m)

In [ ]:
# Some various utility functions that I have been using to check model performance
evaluate_MB_error(m)

print(f'CO2 Capture = {m.CO2_capture():.3}')

# check_scaling(m)

# degen_hunter(m)

# jac, variables, constraints = scaling_script(m)

# Creating a full RPB model

In [ ]:
RPB = full_model_creation(lean_temp_connection=True)
init_routine_1(RPB)
fix_capture_and_solve(RPB, capture=0.9)

In [ ]:
RPB.energy_requirement.ctype == Var

In [ ]:
dir(RPB.energy_requirement)

In [3]:
report(RPB)

,Value,Doc,Fixed
ads.L,3.000000,Bed Length [m],True
ads.D,10.000000,Bed diameter [m],True
ads.w_rpm,1.000000,bed rotational speed [revolutions/min],True
ads.theta,0.500000,Fraction of bed [-],True
des.theta,0.500000,Fraction of bed [-],False
ads.P_in,1.020133,Inlet flue gas pressure [bar],False
ads.P_out,1.013250,Outlet adsorber pressure [bar],True
ads.F_in,48.828237,Inlet adsorber gas flow [mol/s],False
ads.Tg_in,363.000000,Inlet flue gas temperature [K],True
ads.Tx,363.000000,"heat exchange fluid temperature, constant [K]",True


# Save and load model

In [ ]:
# save model
to_json(RPB, fname="RPB_model.json.gz", gz=True, human_read=False)

In [2]:
# create model, load, and solve
RPB = full_model_creation(lean_temp_connection=True)
fix_capture(RPB, capture = 0.9)

from_json(RPB, fname="base case.json.gz", gz=True)

solver = SolverFactory("ipopt")
solver.options = {
    "max_iter": 1000,
    "bound_push": 1e-22,
    # "halt_on_ampl_error": "yes",
}
solver.solve(RPB, tee=True).write()

Ipopt 3.13.2: max_iter=1000
bound_push=1e-22


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
 

In [ ]:
# create model, load results, and then solve
RPB = ConcreteModel()

RPB.ads = RPB_model(mode="adsorption")
RPB.des = RPB_model(mode="desorption")

# fix BCs
RPB.ads.P_in.fix(1.1)
RPB.ads.Tg_in.fix()
RPB.ads.y_in.fix()
RPB.ads.P_out.fix(1.01325)

RPB.des.P_in.fix(1.1)
RPB.des.Tg_in.fix()
RPB.des.y_in.fix()
RPB.des.P_out.fix(1.01325)

# connect rich stream
# unfix inlet loading and temperature to the desorption section
for z in RPB.des.z:
    if z!=0 and z!=1:
        RPB.des.qCO2_in[z].unfix()
        RPB.des.Ts_in[z].unfix()

# add equality constraint equating inlet desorption loading to outlet adsorption loading. Same for temperature.
@RPB.Constraint(RPB.des.z)
def rich_loading_constraint(RPB,z):
    if z==0 or z==1:
        return Constraint.Skip
    else:
        return RPB.des.qCO2_in[z] == RPB.ads.qCO2[z,1]

@RPB.Constraint(RPB.des.z)
def rich_temp_constraint(RPB,z):
    if z==0 or z==1:
        return Constraint.Skip
    else:
        return 1e-2*RPB.des.Ts_in[z] == 1e-2*RPB.ads.Ts[z,1]
    
# connect lean stream
# unfix inlet loading to the adsorption section
for z in RPB.ads.z:
    if z!=0 and z!=1:
        RPB.ads.qCO2_in[z].unfix()

# add equality constraint equating inlet adsorption loading to outlet desorption loading
@RPB.Constraint(RPB.ads.z)
def lean_loading_constraint(RPB,z):
    if z==0 or z==1:
        return Constraint.Skip
    else:
        return RPB.des.qCO2[z,1] == RPB.ads.qCO2_in[z]
    
# turn on initialization factors
RPB.ads.R_MT_solid = 1
RPB.des.R_MT_solid = 1
RPB.ads.R_MT_gas = 1
RPB.des.R_MT_gas = 1
RPB.ads.R_MT_coeff = 1
RPB.des.R_MT_coeff = 1
RPB.ads.R_HT_ghx = 1
RPB.des.R_HT_ghx = 1
RPB.ads.R_HT_gs = 1
RPB.des.R_HT_gs = 1
RPB.ads.R_delH = 1
RPB.des.R_delH = 1

from_json(RPB, fname="RPB_model.json.gz", gz=True)

solver = SolverFactory("ipopt")
solver.options = {
    "max_iter": 1000,
    "bound_push": 1e-22,
    "halt_on_ampl_error": "yes",
}
solver.solve(RPB, tee=True).write()